#### Meghan Shrivastava(meghna3), Mohit Jain(mohitj2), Tafseer Khan(tafseer2)

# Security Analytics - Identify the Attacks

In [1]:
import pandas as pd
import numpy as np

import pyshark
from pandas.io.json import json_normalize
import json
from itertools import islice
import matplotlib.pyplot as plt
import seaborn as sns

from pgmpy.factors.discrete import DiscreteFactor
from pgmpy.models import FactorGraph
from pgmpy.inference import BeliefPropagation

from collections import Counter

C:\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# Task 2 

In [2]:
G = FactorGraph() ## Create FactorGraph object
###############################
#   TODO: Define factor functions
###############################
f_1 = DiscreteFactor(['S1'],[2],[0.85, 0.15]) #This is the prior 'f'
f_2 = DiscreteFactor(['S1','E1'], [2,2], [0.1, 0.2, 0.0, 0.5])

###############################
#   TODO: Add random variables
#         and factor functions 
###############################
G.add_nodes_from(['S1','E1'])  ## Add random variables 
G.add_factors(f_1, f_2)     ## Add factor functions

###############################
#   TODO: Add the edges for random 
#   variables and factor functions
###############################
G.add_edges_from([('S1',f_1), ('S1',f_2), ('E1',f_2)])

### Task 2.2 & 2.3

In [3]:
bp = BeliefPropagation(G)
###############################
#   TODO: Compute the marginal probability
###############################
margin = bp.query(variables = ['S1'], show_progress = False)
print(margin)

+-------+-----------+
| S1    |   phi(S1) |
+=======+===========+
| S1(0) |    0.2550 |
+-------+-----------+
| S1(1) |    0.0750 |
+-------+-----------+


Since S1 represents no attack ($S_1$ = 0), it maximizes P($S_1$)

### Task 2.4 & 2.5

In [4]:
margin = bp.query(variables = ['S1'], evidence = {'E1':1}, show_progress = False)
print(margin)

+-------+-----------+
| S1    |   phi(S1) |
+=======+===========+
| S1(0) |    0.1700 |
+-------+-----------+
| S1(1) |    0.0750 |
+-------+-----------+


$S_1$ = 0 (no attack) is the most probable.

### Task 2.6

<img src="files/task_2.3.jpeg">

# Task 3

In [5]:
ATTACK_EVENTS_MAP = {
    'Scan':1,
    'Login':2,
    'Sensitive_URI':3,
    'New_Kernel_Module':4,
    'DNS_Tunneling':5
}
ATTACK_STATES_MAP = {
    'benign': 1,
    'discovery': 2,
    'access': 3,
    'lateral_movement': 4,
    'privilege_escalation': 5,
    'persistence': 6,
    'defense_evasion': 7,
    'collection': 8,
    'exfiltration': 9,
    'command_control': 10,
    'execution': 11
}

### Task 3.0 

In [6]:
event_review = open('event_review.txt', 'r') 
Lines = event_review.readlines()
df = pd.DataFrame(index = range(len(Lines)), columns = ['Event', 'Latent Attack State'])
i = 0
def fact_func(string):
    x = df.loc[i,'Latent Attack State'] = ((string.split(' ')[-1]).split('\n')[0]).split(']')[-1]
    y = df.loc[i,'Event'] = (string.split(' ')[0]).split(']')[1]
    return x,y
for string in Lines:
    fact_func(string)
    if string.split(' ')[-1] == '\n':
        df.loc[i,'Latent Attack State'] = 'exfiltration'
    i=i+1

df = df.replace({"Event":ATTACK_EVENTS_MAP, "Latent Attack State":ATTACK_STATES_MAP})




In [7]:
def forfunc():
    for i in n_df['Latent Attack State'].unique():
        templist[i-1] = n_df['Latent Attack State'].value_counts()[i] / len(n_df)
    return templist

In [8]:
def window(seq, n=3):                 
    it = iter(seq)
    result = tuple(islice(it, n))
    if len(result) == n:
        yield result
    for elem in it:
        result = result[1:] + (elem,)
        yield result
        
df_dict = {}
for event_ in df['Event'].unique():
    templist = 11*[0]
    n_df = df[df['Event'] == event_]
    forfunc()
    df_dict[event_] = forfunc()
df_dict       

{3: [0.5533333333333333, 0, 0, 0, 0.44666666666666666, 0, 0, 0, 0, 0, 0],
 1: [0.936, 0.064, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 4: [0.875, 0, 0, 0, 0, 0.125, 0, 0, 0, 0, 0],
 5: [0.02, 0, 0, 0, 0, 0, 0, 0, 0.98, 0, 0],
 2: [1.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}

In [9]:
events = [1,2,3,3,3,4,5,5,5] #e1 to e9 as given in the problem
f = [] #A list of lists, each row is the fi for ei
for event in events:
    f.append(df_dict[event])
f

[[0.936, 0.064, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [1.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0.5533333333333333, 0, 0, 0, 0.44666666666666666, 0, 0, 0, 0, 0, 0],
 [0.5533333333333333, 0, 0, 0, 0.44666666666666666, 0, 0, 0, 0, 0, 0],
 [0.5533333333333333, 0, 0, 0, 0.44666666666666666, 0, 0, 0, 0, 0, 0],
 [0.875, 0, 0, 0, 0, 0.125, 0, 0, 0, 0, 0],
 [0.02, 0, 0, 0, 0, 0, 0, 0, 0.98, 0, 0],
 [0.02, 0, 0, 0, 0, 0, 0, 0, 0.98, 0, 0],
 [0.02, 0, 0, 0, 0, 0, 0, 0, 0.98, 0, 0]]

f1 to f9 

### Task 3.1

In [10]:
c_prob, r_prob = [], []
c_count, r_count = Counter(), Counter()
def prob():
    c_prob.append(c)
    r_prob.append(r)
    

In [11]:
def strip():
    l = line.strip().split(' ')
    return l
attack_sequences = open('attack_sequences.txt', 'r') 
Lines = attack_sequences.readlines() 
for line in Lines:
    l = strip()
    for sequence in window(l,3):
        x = len(set(sequence)) 
        if  x== 1:
            r_count[sequence] +=1
        else:
            c_count[sequence] +=1
  

In [12]:
c_sequence = c_count.most_common(1)[0][0]
r_sequence = r_count.most_common(1)[0][0]

for line in Lines:
    l = strip()
    c, r = False, False
    for sequence in window(l,3):
        if sequence == c_sequence:
            c = True
        if sequence == r_sequence:
            r = True
    prob()
    


# len_divide()
c_factor_function, r_factor_function = 11*[0], 11*[0]
cp = sum(c_prob)/len(c_prob)
rp = sum(r_prob)/len(r_prob)  
c_factor_function[5] = cp
r_factor_function[4] = rp


### Task 3.2  You will have to submit the graph you draw through Compass

CP1.5

### Task 3.3

In [13]:
G = FactorGraph() ## Create FactorGraph object
###############################
#   TODO: Define factor functions
###############################
f_1 = DiscreteFactor(['s1','e1'], [11,1], f[0])
f_2 = DiscreteFactor(['s2','e2'], [11,1], f[1])
f_3 = DiscreteFactor(['s3','e3'], [11,1], f[2])
f_4 = DiscreteFactor(['s4','e4'], [11,1], f[3])
f_5 = DiscreteFactor(['s5','e5'], [11,1], f[4])
f_6 = DiscreteFactor(['s6','e6'], [11,1], f[5])
f_7 = DiscreteFactor(['s7','e7'], [11,1], f[6])
f_8 = DiscreteFactor(['s8','e8'], [11,1], f[7])
f_9 = DiscreteFactor(['s9','e9'], [11,1], f[8])

r = DiscreteFactor(['s5','e3','e4','e5'], [11,1,1,1], r_factor_function)
c = DiscreteFactor(['s6','e1','e3','e6'], [11,1,1,1], c_factor_function) 





In [14]:
marginal = 9*[0]

    #First let's analyze all independent nodes:
independent_nodes = [2, 7, 8, 9]

def ind_add(x):
    for node in independent_nodes:
        G = FactorGraph()
        G.add_nodes_from(['s'+str(node), 'e'+str(node)])
        factor_function = DiscreteFactor(['s'+str(node), 'e'+str(node)], [11,1], f[node - 1])
        G.add_factors(factor_function)
        G.add_edges_from([('s'+str(node), factor_function), ('e'+str(node), factor_function)])
        if x==1:
            marginal[node - 1] =  BeliefPropagation(G).query(variables = ['s'+str(node)], show_progress = False)
        if x==0:    
            marginal_without_7[node - 1] =  BeliefPropagation(G).query(variables = ['s'+str(node)], show_progress = False)

ind_add(1)

G = FactorGraph()
dependent_nodes = [1,3,4,5,6]

def dep_add():
    G.add_nodes_from(['s'+str(i) for i in dependent_nodes] + ['e'+str(i) for i in dependent_nodes])  ## Add random variables 
    G.add_factors(f_1,f_3,f_4,f_5,f_6,r,c)
    G.add_edges_from([('s1',f_1), ('e1', f_1), ('s3',f_3), ('e3', f_3), ('s4',f_4), ('e4', f_4),('s5',f_5), ('e5', f_5), ('s6',f_6), ('e6', f_6),('s5',r), ('e3', r), ('e4', r), ('e5', r),
                      ('s6',c), ('e1', c), ('e3', c), ('e6', c)])
dep_add()
bp = BeliefPropagation(G)
for node in dependent_nodes:
    marginal[node - 1] = bp.query(variables = ['s'+str(node)], show_progress = False)

### Task 3.4

#### a. At every time point, provide the marginal probability of each state (Since we have 9 time points and 11 possible states, you should provide 99 probability values here)

In [15]:
marginal[0].normalize()
print(marginal[0])


+--------+-----------+
| s1     |   phi(s1) |
+========+===========+
| s1(0)  |    0.9360 |
+--------+-----------+
| s1(1)  |    0.0640 |
+--------+-----------+
| s1(2)  |    0.0000 |
+--------+-----------+
| s1(3)  |    0.0000 |
+--------+-----------+
| s1(4)  |    0.0000 |
+--------+-----------+
| s1(5)  |    0.0000 |
+--------+-----------+
| s1(6)  |    0.0000 |
+--------+-----------+
| s1(7)  |    0.0000 |
+--------+-----------+
| s1(8)  |    0.0000 |
+--------+-----------+
| s1(9)  |    0.0000 |
+--------+-----------+
| s1(10) |    0.0000 |
+--------+-----------+


#### b. At every time point, provide the most probable state

In [16]:
# The most probable states at each time point are:

print("t= 1: benign")

print ("t = 2: benign")

print ("t = 3: benign")

print ("t = 4: benign")

print ("t = 5: privilege escalation")

print ("t = 6: persistence")

print ("t  7: exfiltration")

print ("t = 8: exfiltration")

print ("t = 9: exfiltration")

t= 1: benign
t = 2: benign
t = 3: benign
t = 4: benign
t = 5: privilege escalation
t = 6: persistence
t  7: exfiltration
t = 8: exfiltration
t = 9: exfiltration


### Task 3.5

In [17]:
ACTIONS = {
    # each value in an actions' vector corresponds to an attack stage
    'NO-OP':   [1.,   0.61, 0.69, 0.09, 0.2 , 0. ,  0.,   0.,   0. ,  0. ,  0.  ],
    'MONITOR': [0.  , 0.39, 0.31 ,0.84, 0.63, 0.7,  0.07 ,0.1 , 0. ,  0. ,  0.  ],
    'STOP':    [0.  , 0.,   0.  , 0.07, 0.17, 0.3,  0.93 ,0.9 , 1. ,  1. ,  1.  ]
}

hidden_states = ['benign', 'benign', 'benign', 'benign', 'privilege_escalation', 'persistence','exfiltration', 'exfiltration', 'exfiltration']
hidden_states_mapped = [ATTACK_STATES_MAP[state] for state in hidden_states]
    

actions = []
def actn():
    temp = np.argmax(np.array(list(ACTIONS.values())), axis = 0)
    for i in hidden_states_mapped:
        actions.append(list(ACTIONS.keys())[temp[i-1]])
    print(actions)
actn()    

['NO-OP', 'NO-OP', 'NO-OP', 'NO-OP', 'MONITOR', 'MONITOR', 'STOP', 'STOP', 'STOP']


### Task 3.6

#### Indicate the earliest stage in which your model should recommend stopping the attack

'STOP' is recommended in S7

### Task 3.7

#### a. Judge whether the most probable states for $s_1-s_6,s_8,s_9$ remain the same as Task3.2


In [18]:
f_1 = DiscreteFactor(['s1','e1'], [11,1], f[0])
f_2 = DiscreteFactor(['s2','e2'], [11,1], f[1])
f_3 = DiscreteFactor(['s3','e3'], [11,1], f[2])
f_4 = DiscreteFactor(['s4','e4'], [11,1], f[3])
f_5 = DiscreteFactor(['s5','e5'], [11,1], f[4])
f_6 = DiscreteFactor(['s6','e6'], [11,1], f[5])
f_8 = DiscreteFactor(['s8','e8'], [11,1], f[7])
f_9 = DiscreteFactor(['s9','e9'], [11,1], f[8])

r = DiscreteFactor(['s5','e3','e4','e5'], [11,1,1,1], r_factor_function)
c = DiscreteFactor(['s6','e1','e3','e6'], [11,1,1,1], c_factor_function) 




In [19]:
marginal_without_7 = 9*[0]

independent_nodes = [2, 8, 9]


ind_add(0)
    

G = FactorGraph()
dependent_nodes = [1,3,4,5,6]


dep_add()

bp = BeliefPropagation(G)
for node in dependent_nodes:
    marginal_without_7[node - 1] = bp.query(variables = ['s'+str(node)], show_progress = False)

for i in range(9):
    try:
        marginal_without_7[i].normalize()
    except AttributeError:
        pass
    print(marginal_without_7[i] == marginal[i])

True
True
False
False
False
False
False
True
True


Marginals remain same except S7 which has been removed.

#### b. State the reason for your judgement



S7 and E7 are independent of other nodes. So the value of S7 does not affect the rest of the graph.

### Task 3.8

#### a. Draw an HMM model for the attack scenario given the provided states and events.

<img src="files/task_3.8.jpeg">


#### b. What parameters are needed for this HMM model to work?

Observation matrix, State transition matrix and priors.


#### c. Give an example of an advantage of the FG over the HMM model.

The HMMM model works on the assumption of Markov property whereas the factor graph is not restricted by any such assumption. It is a straightforward approach where it is easier to consider multiple relationships between nodes (using r and c) 

# Task 4

### Task 4.0
#### Is it possible to 100% detect this attack using only one event, e.g., ϵ1, of the six listed events? Briefly explain.
No. Since there are 6 listed events associated with the attack term "benign", in any possible scenario we cannot just frame our assumptions based on just one event as each event has a probability associated with it.

### Task 4.1
#### For each of the six listed events, give an example of a situation in which a false positive (i.e., mis-detecting a legitimate user as an attacker) could happen.
Scan: System admin is doing a scan.

Login: False positive not possible because it maps to just to one state=benign.

Sensitive URI: There is a possibility that a false positive can be generated in this case as the user can access the executable URI. That's beacuse each sensitive URI points to an executable which does not necessarily indicate coming from the server related to the attacker. 

New Exe File: There is a possibility that a false positive can be generated in this case as the user can access the exe file. That's because each exe file points to an executable which does not necessarily indicate coming from the server related to the attacker.

Homepage Overwritten with new link: User has the authority to override the homepage with a new link. So we cannot always indicate that the file is through the attackers server. 

Webserver Restarted: User has the authority to override the webserver. So we cannot always indicate that the file is through the attackers server.

### Task 4.2
#### Provide a visual representation of a factor graph that can model the attack described above.
<img src="files/task_4.2.jpeg">

### Task 4.3
#### What variables and factor functions are common to the factor graph in Task 3 and your factor graph in 4.2?


Events that are common in both the cases and mapping to the same states are:
i) Login-> {benign}
ii) Scan-> {benign, discovery}
iii) Sensitive URI-> {benign, privilege escalation}



